## Finding images

### Image filters
`Image` searches support a special method `intersects()` which is used to filter images by means of a geospatial search. Unlike `filter()` this method cannot be used multiple times. It will accept as an argument a GeoJSON dictionary, a shapely geometry, or any of the DL standard `GeoContext` object types. It will select any image for which the image geometry intersects the supplied geometry in lat-lon space (i.e. WGS84). As coordinate system transformations of bounding boxes are involved here, it should be noted that this filtering can be inexact; the overlap of geometries in the native coordinate system of the image may not be the same as that when transformed to the geographic coordinate system.

Please see the `GeoContext Guide` for more information about working with `GeoContexts`.

Please consult the API documentation for the `Image` class for information on which properties can be filtered.

Search images by the most common attributes - by product, intersecting with a geometry and by a date range:

In [ ]:
from descarteslabs.catalog import Image, Product, properties as p

geometry = {
    "type": "Polygon",
    "coordinates": [
        [
            [2.915496826171875, 42.044193618165224],
            [2.838592529296875, 41.92475971933975],
            [3.043212890625, 41.929868314485795],
            [2.915496826171875, 42.044193618165224],
        ]
    ],
}

search = Product.get("usgs:landsat:oli-tirs:c2:l2:v0").images()
search = search.intersects(geometry)
search = search.filter("2023-01-01" <= p.acquired < "2023-06-01")
search = search.sort("acquired")
search.count()

There are other attributes useful to filter by, documented in the API reference for [`Image`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/image.html#descarteslabs.catalog.Image). For example exclude images with too much cloud cover:

In [ ]:
search = search.filter(p.cloud_fraction < 0.2)
search.count()

Filtering by `cloud_fraction` is only reasonable when the product sets this attribute on images. `Images` that don’t set the attribute are excluded from the filter.

The `created` timestamp is added to all objects in the catalog when they are created and is immutable. Restrict the search to results created before some time in the past, to make sure that the image results are stable:

In [ ]:
from datetime import datetime

search = search.filter(p.created < datetime(2023, 4, 1))
search.count()

Note that for all timestamps we can use `datetime` instances or strings that can reasonably be parsed as a timestamp. If a timestamp has no explicit timezone, it’s assumed to be in UTC.

## ImageCollections
We can use the `collect()` method with an image search to obtain an `ImageCollection` with many useful features:

In [ ]:
images = search.collect()
images

Our original AOI for the search is available on the image collection:

In [ ]:
images.geocontext

We can extract attributes across the collection with `each()`, or filter or group based on their attributes with `filter()` and `groupby()`:

In [ ]:
list(images.each.acquired.month)

In [ ]:
spring = images.filter(lambda i: 3 <= i.acquired.month < 6)
list(spring.groupby(lambda i: i.acquired.month))

## Image summaries
Any queries for images support a summary via the `summary()` method, returning a `SummaryResult` with aggregate statistics beyond just the number of results:



In [ ]:
from descarteslabs.catalog import Image, properties as p

search = Image.search().filter(p.product_id == "usgs:landsat:oli-tirs:c2:l2:v0")
search.summary()

These summaries can also be bucketed by time intervals with `summary_interval()` to create a time series:

In [ ]:
search.summary_interval(
    interval="month", start_datetime="2023-01-01", end_datetime="2023-06-01"
)